# Content Words

In [ ]:
!pip install pymprog 

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 2.1MB 5.8MB/s 
  Created wheel for pymprog: filename=pymprog-1.1.2-cp37-none-any.whl size=43502 sha256=44c78b4dd67cd741f1b71cb4fca033c18310fbccf87024ad3304732f1ec1e41e
  Stored in directory: /root/.cache/pip/wheels/fa/09/59/ef649891969b65ff8de663e7a0a65bf27ba0c273e2a0795a44
Successfully built pymprog


In [ ]:
!pip install nltk

!pip install spacy==2.3.5

!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

!pip install pyresparser

     |████████████████████████████████| 12.1MB 324kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-cp37-none-any.whl size=12047109 sha256=51e14b5dcdf9e3dcce20c0534cf1470803d7c9a05e7aaa1c996e7b14ad2d2bd3
  Stored in directory: /root/.cache/pip/wheels/2b/3f/41/f0b92863355c3ba34bb32b37d8a0c662959da0058202094f46
Successfully built en-core-web-sm


In [ ]:

import spacy



In [ ]:

from __future__ import division



import numpy as np 
import pandas as pd 
import spacy
from spacy.tokens.doc import Doc
import inspect
#import textacy 

#from textacy.vsm import Vectorizer
#import textacy.vsm


import scipy.sparse as sp
import tqdm as tqdm

import re

In [ ]:
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading the data

In [ ]:
train = pd.read_csv('/content/drive/My Drive/AIR/train.csv', encoding = 'ISO-8859-1')
test = pd.read_csv('/content/drive/My Drive/AIR/test.csv', encoding = 'ISO-8859-1')
print(len(train),len(test))

7613 3263


In [ ]:
for col in train.columns:
        print(col , train[col].isna().sum())

id 0
keyword 61
location 2533
text 0
target 0


In [ ]:
for col in test.columns:
        print(col , test[col].isna().sum())

id 0
keyword 26
location 1105
text 0


In [ ]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
for df in [train , test]:
    for e in ['keyword', 'location']:
        df[e] = df[e].fillna('')

## Preprocessing

For my tweets to be informative, there are a few terms I can immediately remove. For instance, any urls won't be useful to the rescue teams. Equally, any '@...' are just calling another twitter handle, and are equally not useful. 

In [ ]:
def Clean(text):
    
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text) #remove URL
    
    text=text.replace(r'&amp;?',r'and')
    text=text.replace(r'&lt;',r'<')
    text=text.replace(r'&gt;',r'>') #remove special character 
    
    text = re.sub(r"(?:\@)\w+", '', text)
    text=text.encode("ascii",errors="ignore").decode()
    text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
    
    text=re.sub(r'[!]+','!',text)
    text=re.sub(r'[?]+','?',text)
    text=re.sub(r'[.]+','.',text)
   
    text=re.sub(r"'","",text)
    text=re.sub(r"\(","",text)
    text=re.sub(r"\)","",text)
    
    text=" ".join(text.split())
    return text

def stemmer(text):
    stem_text = [PorterStemmer().stem(i) for i in text]
    return stem_text

def lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

train['text']=train['text'].apply(lambda x: Clean(x))
test['text']=test['text'].apply(lambda x: Clean(x))


In [ ]:
train.head(-5)

,id,keyword,location,text,target
0,1,,,Our Deeds are the Reason of this earthquake Ma...,1
1,4,,,Forest fire near La Ronge Sask Canada,1
2,5,,,All residents asked to shelter in place are be...,1
3,6,,,13000 people receive wildfires evacuation orde...,1
4,7,,,Just got sent this photo from Ruby Alaska as s...,1
...,...,...,...,...,...
7603,10862,,,Officials say a quarantine is in place at an A...,1
7604,10863,,,WorldNews Fallen powerlines on Glink tram UPDA...,1
7605,10864,,,on the flip side Im at Walmart and there is a ...,1
7606,10866,,,Suicide bomber kills 15 in Saudi security site...,1


In [ ]:
test.head()

,id,keyword,location,text
0,0,,,Just happened a terrible car crash
1,2,,,Heard about earthquake is different cities sta...
2,3,,,there is a forest fire at spot pond geese are ...
3,9,,,Apocalypse lighting Spokane wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
data = [test["text"]]
headers = ["text"]
tweets = pd.concat(data, axis=1, keys=headers)
tweets.head()

,text
0,Just happened a terrible car crash
1,Heard about earthquake is different cities sta...
2,there is a forest fire at spot pond geese are ...
3,Apocalypse lighting Spokane wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


There are alot of `u'RT'` terms in the tweet texts. Since this isn't a word, SpaCy doesn't know how to handle them. Since these add nothing to the content of a tweet, I'm just going to get rid of them. 

In [ ]:
len(tweets)

3263

Tokenizing with SpaCy

In [ ]:
nlp = spacy.load('en_core_web_sm')#default pipeline package

In [ ]:
spacy_tweets = []

for doc in nlp.pipe(tweets.text, n_threads = -1):
    spacy_tweets.append(doc)

In [ ]:
print(len(spacy_tweets))
spacy_tweets[1801]

3263


Good morning Slept about 6 hours The heat wave hits Japan The hot days still continue I guess some people were killed by this climate

Not all tweets are equally useful. Some just contain prayers, such as

`Hope it doesn't rain. #Nepal`

whereas others are dense with useful information: 

`2 Dead, 100 Injured in Bangladesh From Nepal Quake`

How do I decide which parts of these tweets are most useful? One way to do it is to measure the term frequency-inverse document frequency (tf-idf) of each of the words in the corpus of tweets. This metric measures how important a word is in a corpus of tweets. 

## Getting the tf-idf values of content words. 

I can do a preliminary 'cleanup', by keeping only 'content words'. These are defined as : Numerals, Nouns and Verbs. Conveniantly, SpaCy has already organised this for us. 

In [ ]:
spacy_tweets[90]

I care most about tokens which are entities, and numbers. The other tokens have too much noise, so let's focus on these two:

In [ ]:
target_1=train[train["target"] == 1]
target_1=target_1[target_1["keyword"] != '']
main_words = list(set(target_1["keyword"]))
print(len(main_words))
main_words = ['\\'+"u"+s for s in main_words] #adding unicode escape sequence
main_words

220


['\\uflood',
 '\\ubody%20bags',
 '\\uquarantine',
 '\\uburning%20buildings',
 '\\uweapon',
 '\\ubridge%20collapse',
 '\\usuicide%20bombing',
 '\\uscreamed',
 '\\ublood',
 '\\uelectrocuted',
 '\\urioting',
 '\\uattacked',
 '\\urescue',
 '\\uelectrocute',
 '\\utwister',
 '\\uruin',
 '\\ueyewitness',
 '\\ucyclone',
 '\\uloud%20bang',
 '\\ucatastrophic',
 '\\uwhirlwind',
 '\\uthunder',
 '\\unuclear%20reactor',
 '\\usnowstorm',
 '\\utraumatised',
 '\\uengulfed',
 '\\uhijack',
 '\\uoil%20spill',
 '\\uthunderstorm',
 '\\uwildfire',
 '\\ubattle',
 '\\ufatalities',
 '\\udisaster',
 '\\uemergency%20services',
 '\\ucliff%20fall',
 '\\udetonation',
 '\\uradiation%20emergency',
 '\\uexplosion',
 '\\uheat%20wave',
 '\\udevastation',
 '\\ublizzard',
 '\\udeluge',
 '\\urefugees',
 '\\ubloody',
 '\\uinjuries',
 '\\uwounded',
 '\\ubioterror',
 '\\uarmageddon',
 '\\uderailed',
 '\\uambulance',
 '\\ucatastrophe',
 '\\uhurricane',
 '\\ubush%20fires',
 '\\uhazardous',
 '\\udrought',
 '\\udust%20storm',
 '\\

In [ ]:
useful_entities = [u'NORP', u'FACILITY', u'ORG', u'GPE', u'LOC', u'EVENT', u'DATE', u'TIME']

In [ ]:
content_tweets = []
for single_tweet in tqdm.tqdm(spacy_tweets):
    single_tweet_content = []
    for token in single_tweet: 
        if ((token.ent_type_ in useful_entities)  
            or (token.pos_ == u'NUM') 
            or (token.lower_ in main_words)):
            single_tweet_content.append(token)
    content_tweets.append(single_tweet_content)

100%|██████████| 3263/3263 [00:00<00:00, 15033.15it/s]


In [ ]:
tweet_num = 150
print ("original_tweet \n" + str(spacy_tweets[tweet_num]) 
       + "\n\noriginal_tweet\n" + str([str(x) for x in spacy_tweets[tweet_num]])
       + "\n\ncontent_tweet\n" + str(content_tweets[tweet_num])
      )

original_tweet 
You da One MTVSummerStar VideoVeranoMTV MTVHottest Britney Spears Lana Del Rey

original_tweet
['You', 'da', 'One', 'MTVSummerStar', 'VideoVeranoMTV', 'MTVHottest', 'Britney', 'Spears', 'Lana', 'Del', 'Rey']

content_tweet
[One]


So this has already gone some way to (crudely) isolating the interesting parts of a tweet. 

Unfortunately, SpaCy doesn't calculate tf-idf score automatically. There IS a library which can do this: [textacy](https://textacy.readthedocs.io/en/latest/index.html). Note: textacy is built on SpaCy.

I care about the tf-idf scores of the entire tweet, so will find the tf-idf score across the entire corpus of original tweets. 

In [ ]:
vectorizer = Vectorizer(weighting = 'tfidf')

NameError: ignored

To calculate the tf-idf score of all the tokens in the tweets, I can use `fit_transform()`. 

Note: I am using the `lemma_` attribute of each token, because tokens contain information about the documents. This means that 'Nepal' in the 100th tweet will have a different **token** from 'Nepal' in the 200th tweet, but the same `lemma__` attribute. This is what I want to compare - I don't want hundreds of 'Nepal' columns in my term matrix. 

In [ ]:
term_matrix = vectorizer.fit_transform([tok.lemma_ for tok in doc] for doc in spacy_tweets)

This matrix is a term-document matrix. What this means is that on top of having the tf-idf values, each row is a document (and each column is a word). 

If the tweet in row `i` contains the column in row `j`, then the element `matrix[i][j]` will contain the tf-idf value. If the tweet *doesn't* contan the word, the matrix value will be zero. 

In [ ]:
np_matrix = term_matrix.todense()

In [ ]:
np_matrix.shape

My ultimate goal is to create a dictionary, which maps from the tokens in the content tweets to some tf-idf score. To do this, I need to find out which tokens are at what columns in the term matrix. 

The vectorizer object has a dictionary, which maps token.lemma_ to its column. 

In [ ]:
for key in sorted(vectorizer.vocabulary)[1000:1015]:
    print key, vectorizer.vocabulary[key]

And each column (word) has a unique tf-idf value.

I can therefore map the value of the content tokens to their tf-idf, using the `vectorizer.vocabulary` dictionary. 

In [ ]:
for token in content_tweets[500]:
    print (token.lemma_, vectorizer.vocabulary[token.lemma_], np.max(np_matrix[:,vectorizer.vocabulary[token.lemma_]]))

In [ ]:
tfidf_dict = {}
content_vocab = []
for tweet in content_tweets: 
    for token in tweet: 
        if token.lemma_ not in tfidf_dict: 
            content_vocab.append(token.lemma_)
            tfidf_dict[token.lemma_] = np.max(np_matrix[:,vectorizer.vocabulary[token.lemma_]])

In [ ]:
for key in sorted(tfidf_dict)[500:505]:
    print ("WORD:" + str(key) + " -- tf-idf SCORE:" +  str(tfidf_dict[key]))

Success! 

## COntent Word-based Tweet Summarization (COWTS) 
As per [Rudra et al](http://dl.acm.org/citation.cfm?id=2806485). 

I'll be using [PyMathProg](http://pymprog.sourceforge.net/index.html) as my Integer Linear Programming Solver. This is a python interface for [GLPK](https://www.gnu.org/software/glpk/)

In [ ]:
import pymprog 

ModuleNotFoundError: ignored

I want to maximize 
\begin{equation}
\sum_{i=1}^n x_{i} + \sum_{j = 1}^{m} Score(j) \cdot y_{j}
\end{equation}
Where $x_{i}$ is 1 if I include tweet i, or 0 if I don't, and where $y_{j}$ is 1 or 0 if each content word is included (and Score(j) is that word's tf-idf score). 

I'm going to subject this equation to the following constraints: 

1. 
\begin{equation}
\sum_{i=1}^{n} x_{i} \cdot Length(i) \leq L
\end{equation}
I want the total length of all the selected tweets to be less than some value L, which will be the length of my summary, L. I can vary L depending on how long I want my summary to be. 

2. 
\begin{equation}
\sum_{i \in T_{j}} x_{i} \geq y_{j}, j = [1,...,m]
\end{equation}
If I pick some content word $y_{j}$ (out of my $m$ possible content words) , then I want to have at least one tweet from the set of tweets which contain that content word, $T_{j}$. 

3. 
\begin{equation}
\sum_{j \in C_{i}} y_{j} \leq |C_{i}| \times x_{i}, i = [1,...,n]
\end{equation}
If I pick some tweet i (out of my $n$ possible tweets) , then all the content words in that tweet $C_{i}$ are also selected. 

In [ ]:
begin('COWTS')

NameError: ignored

In [ ]:
# Defining my first variable, x 
# This defines whether or not a tweet is selected
x = var('x', len(spacy_tweets), bool)

# Check this worked
x[1000]

In [ ]:
# Also defining the second variable, which defines
# whether or not a content word is chosen
y = var('y', len(content_vocab), bool)

In [ ]:
len(y), y[0]

Now that I have defined my variables, I can define the equation I am maximizing. 

In [ ]:
maximize(sum(x) + sum([tfidf_dict[content_vocab[j]]*y[j] for j in range(len(y))]));

Now, I can define my constraints. First, 
\begin{equation}
\sum_{i=1}^{n} x_{i} \cdot Length(i) \leq L
\end{equation}

In [ ]:
## Maximum length of the entire tweet summary

# Was 150 for the tweet summary, 
# But generated a 1000 word summary for CONABS
L = 1000

# hiding the output of this line since its a very long sum 
sum([x[i]*len(spacy_tweets[i]) for i in range(len(x))]) <= L;

These next two constraints are slightly more tricky, as I need a way to define which content words are in which tweets. 

However, the term matrix I defined using the vectorizer has all of this information. 

I'll begin by defining two helper methods

In [ ]:
def content_words(i):
    '''Given a tweet index i (for x[i]), this method will return the indices of the words in the 
    content_vocab[] array
    Note: these indices are the same as for the y variable
    '''
    tweet = spacy_tweets[i]
    content_indices = []
    
    for token in tweet:
        if token.lemma_ in content_vocab:
            content_indices.append(content_vocab.index(token.lemma_))
    return content_indices

In [ ]:
def tweets_with_content_words(j):
    '''Given the index j of some content word (for content_vocab[j] or y[j])
    this method will return the indices of all tweets which contain this content word
    '''
    content_word = content_vocab[j]
    
    index_in_term_matrix = vectorizer.vocabulary[content_word]
    
    matrix_column = np_matrix[:, index_in_term_matrix]
    
    return np.nonzero(matrix_column)[0]

I can now define the second constraint: 
\begin{equation}
\sum_{i \in T_{j}} x_{i} \geq y_{j}, j = [1,...,m]
\end{equation}

In [ ]:
for j in range(len(y)):
    sum([x[i] for i in tweets_with_content_words(j)])>= y[j]

And the third constraint:
\begin{equation}
\sum_{j \in C_{i}} y_{j} \leq |C_{i}| \times x_{i}, i = [1,...,n]
\end{equation}

In [ ]:
for i in range(len(x)):
    sum(y[j] for j in content_words(i)) >= len(content_words(i))*x[i]

In [ ]:
solve()

In [ ]:
result_x =  [value.primal for value in x]
result_y = [value.primal for value in y]

In [ ]:
end()

In [ ]:
chosen_tweets = np.nonzero(result_x)
chosen_words = np.nonzero(result_y)

In [ ]:
len(chosen_tweets[0]), len(chosen_words[0])

Lets take a look at the results! 

In [ ]:
for i in chosen_tweets[0]:
    print ('--------------')
    print spacy_tweets[i]

There is definitely noise amongst these tweets, but these tweets do successfully provide a good overview of the situation in Nepal.

I am going to compare this to random tweets, to make sure it does perform better than 16 randomly chosen tweets. 

In [ ]:
random_tweets = np.random.choice(spacy_tweets, size=11)

In [ ]:
for i in random_tweets:
    print ('--------')
    print i

A brief comparison does indicate that this method is far better than random choice at providing a situational overview. 

It's worth noting that even a random distribution will contain a fair amount of information, because of the selective nature in which we isolated tweets; this is already a subsample which contains a higher % of relevant information. 

In [ ]:
cowts_tweets = []
for i in chosen_tweets[0]:
    cowts_tweets.append(spacy_tweets[i])

Lets take a look at the first few tweets

In [ ]:
for tweet in cowts_tweets[:10]:
    print ('--------')
    print tweet

This notebook is getting long, so I'm going to save these tweets (which I will continue using) and start a fresh notebook for the next steps. 

## Saving everything for a fresh notebook 

In [ ]:
cowts_unicode = [x.text for x in cowts_tweets]

In [ ]:
cowts_dataframe = pd.DataFrame(cowts_unicode)

In [ ]:
cowts_dataframe.head()

,0
0,: LATEST Nepal's Kantipur TV shows at least 21...
1,Prayers for the affected people across SouthAs...
2,: Due to bulding collaps 12 People died in Eas...
3,": M7.9 - 29km ESE of Lamjung, Nepal 20 00 29..."
4,"Earthquake: 2015-04-25 17:30HKT M5.0 [28.0N,85..."


Saving it to a pickle: 

In [ ]:
cowts_dataframe.to_pickle('cowts_tweets.pkl')

In [ ]:
np.save('term_matrix.npy', np_matrix)

In [ ]:
np.save('tweet_indices.npy', chosen_tweets)

In [ ]:
np.save('vocab_to_idx.npy', vectorizer.vocabulary)

In [ ]:
np.save('content_vocab.npy', content_vocab)

In [ ]:
np.save('tfidf_dict.npy', tfidf_dict)